In [ ]:
#@title Initial setup, run once
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

import math
from torch.utils.data import Dataset

!wget https://raw.githubusercontent.com/LadMikkie/vnet/v-moderu
!wget https://raw.githubusercontent.com/LadMikkie/vnet/v-datasetto


class CharDataset(Dataset):

    def __init__(self, data, block_size):
        chars = list(set(data))
        data_size, vocab_size = len(data), len(chars)
        print('data has %d characters, %d unique.' % (data_size, vocab_size))
        
        self.stoi = { ch:i for i,ch in enumerate(chars) }
        self.itos = { i:ch for i,ch in enumerate(chars) }
        self.block_size = block_size
        self.vocab_size = vocab_size
        self.data = data
    
    def __len__(self):
        return math.ceil(len(self.data) / (self.block_size + 1))

    def __getitem__(self, idx):
        # we're actually going to "cheat" and pick a spot in the dataset at random
        i = np.random.randint(0, len(self.data) - (self.block_size + 1))
        chunk = self.data[i:i+self.block_size+1]
        dix = [self.stoi[s] for s in chunk]
        x = torch.tensor(dix[:-1], dtype=torch.long)
        y = torch.tensor(dix[1:], dtype=torch.long)
        return x, y

model = torch.load("v-moderu", map_location="cuda")
train_dataset = torch.load("v-datasetto")


Mounted at /content/drive
/content/drive/MyDrive/voreNET


## voreNET

In [10]:
#@title Enter your request
context = "" #@param {type:"string"}
max_length = 512 #@param {type:"slider", min:1, max:512, step:1}
temperature = 0.8 #@param {type:"slider", min:0.01, max:1, step:0.01}
top_k = 5 #@param {type:"slider", min:1, max:231, step:1}



from mingpt.utils import sample

x = torch.tensor([train_dataset.stoi[s] for s in context], dtype=torch.long)[None,...].to("cuda")
y = sample(model, x, max_length, temperature=temperature, sample=True, top_k=top_k)[0]
completion = ''.join([train_dataset.itos[int(i)] for i in y])
completion = completion.replace("â", "'")
completion = completion.replace("â", "\n")
completion = completion.replace("'", "\"")
completion = completion.replace("'", "'")
completion = completion.replace("                ", "")
print(completion)

IndexError: ignored